---
title: "2023_Kronensicherung_Plesse_LS3"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Kraefte_LS3
## LS3: Daten der Kraftmessdosen (lineScale3 Data = LS3)

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment) und installier das Paket linescale3 (LS3) inklusive kj_core und kj_logger und weiteren requirements.

## Arbeitsumgebung vorbereiten

Es werden zuerst benötigte Standard-Pakete importiert. Nachfolgend das extra geschriebenen Pakete LS3. Fehler beim Import dieses Pakets sind ggf. Bugs. Es nutzte eine gemeinsame CodeBasis in den Paketen kj_core (Core-Package) und kj_logger (individualisiertes Logging des Verarbeitungs-Prozesses). Diese sollte i. d. R. über die requirements mit installiert werden.

### IMPORT: Importieren von Standardbibliotheken

Die folgenden Bibliotheken werden importiert, um grundlegende Funktionen für Strukturierung, Datenverarbeitung, Plotting und statistische Auswertung bereit zu stellen.

In [1]:
# Struktur
from pathlib import Path
from typing import Dict, List
import json
from IPython.display import Markdown, display

# Datenverarbeitung
import numpy as np
import pandas as pd

### IMPORT: Importieren eignes Packet lineScale3

Das Paket linescale3 wurde vom Autor (Kyell Jensen) zum einfachen Analysieren, Plotten und zur Interpretation der CSV-Messdaten der lineScale3 Kraftmessdose der Firma lineGrip geschrieben (https://www.linegrip.com/linescale-3/). Das Packet ist auf GitHub verfügbar (https://github.com/kyellsen/linescale3). Nachfolgend wird das Packet und einige dort definierten Klassen importiert.

In [2]:
import linescale3 as ls3
# ls3.help() # Test

### IMPORT: Projekt Konfiguration laden

Lege Pfade für Daten-Importe, Daten-Exporte etc. fest (ggf. anpassen an eigene Verzeichnisstruktur), ausgelagert in gemeinsame Config für verschiedene Notebooks

In [3]:
# Importiere alle Einstellungen aus der project_config.py
from project_config import (
    main_path,
    analyse_name,
    data_path,
    working_directory,
    data_export_directory,
    latex_export_directory
)

Lade allgemeine Export-Funktionen, um die Daten als Latex-Tabellen zu exportieren

In [4]:
from utils.latex_export import (
    save_latex_table,
    build_data_dict_df
)

## IMPORT: lineScale Daten laden

Die Funktion 'ls3.setup' erstellt Instanzen spezifischer Klassen, die für das Paket notwendig sind (CONFIG, LOG_MANAGER, PLOT_MANAGER). 

Über die Klasse 'ls3.Series' wird eine neue Messreihe initialisiert und als 'ls3_series' gespeichert. Im Verzeichnis 'ls3_data_path' finden sich die Messdaten von zwei lineScale3 ('LS14BFE6' und 'LS14991E').
LS14BFE6: Befand sich während der Messung im Zugseil zwischen den Stämmlingen und erfasste die Vorspannung. Es wurden insgesamt 31 Logs (hier Measurements) aufgezeichnet. Eine Messfrequenz von 40 Hz erschien hier ausreichend.
LS14991E: Befand sich während der Messung in der Kronensicherung und hat die Kraft in der Kronensicherung nach dem Release gemessen. Es wurden 19 Logs erfasst, da die ersten Messungen als Referenz ohne Kronensicherung stattfanden. Hier wurde die maximale Messfrequenz von 1028 Hz gewählt, um Lastspitzen optimal zu erfassen.

In [5]:
ls3.setup(working_directory=working_directory, log_level="info", safe_logs_to_file=True)

ls3_data_path = data_path / 'LS3'
ls3_series = ls3.classes.Series(name=analyse_name, path=ls3_data_path)

2025-03-27 11:14:35 [INFO] kj_logger.update_config: <kj_logger.LogManager object at 0x000001BC687EAED0> initialized - update_config! Code: 000
2025-03-27 11:14:35 [INFO] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ls3 was successfully created.
2025-03-27 11:14:35 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\ls3!
2025-03-27 11:14:35 [INFO] kj_core.core_config.__init__: <linescale3.config.Config object at 0x000001BC68819650> initialized! Code: 001
2025-03-27 11:14:35 [INFO] linescale3.setup: ls3: Setup linescale3 package!
2025-03-27 11:14:35 [INFO] kj_core.utils.plot_manager.__init__: <kj_core.utils.plot_manager.PlotManager object at 0x000001BC698822D0> initialized! Co

## ANALYSE: Explorative Datenanalyse

### PLOTTING: Daten der Forcemeter

Da teilweise für jede Messung und jeden Sensor ein Plot generiert wird, werden diese nicht im Notebook direkt angezeigt, sondern im Explorer gespeichert. Die Plots liegen entsprechend im working_directory in den Verzeichnissen ls3/plots/ bzw. ptq/plots.

'ls3_series.plot_force_vs_time' erstellt für jede Messung für beide eingesetzten LineScales einen Plot, in dem die ermittelte Kraft (force in kN) gegenüber der Zeit (sec_since_start) abgebildet wird. Die Plots werden im Verzeichnis 'ls3/plots/force_vs_time_1/' abgelegt.

14:BF:E6: Hat die tatsächliche Vorspannung ermittelt und hat entsprechend für alle 29 (-2) Messungen (experiment_id) einen Log aufgezeichnet.
Aufgrund der Messtechnik wurden 2 Logs erstellt, die keine Messung darstellen. Diese dürfen nicht mit ausgewertet werden (Measurement ID 14 und 30).

14:99:1E: Hat die Lasten in der Kronensicherung aufgezeichnet und hat entsprechend nur für die 19 (-1) Messungen mit Kronensicherungen einen Log aufgezeichent.

In [6]:
ls3_series.plot_force_vs_time_with_max_and_release()

2025-03-27 11:14:37 [INFO] linescale3.classes.measurement.plot_force_vs_time_with_max_and_release: plot_force_vs_time_with_max_and_release for measurement: 'Measurement(id: '1', name: '14991E_23_03_22_14_41_10', sensor_id: '14:99:1E')'.
2025-03-27 11:14:38 [INFO] linescale3.classes.measurement.plot_force_vs_time_with_max_and_release: plot_force_vs_time_with_max_and_release for measurement: 'Measurement(id: '2', name: '14991E_23_03_22_14_47_59', sensor_id: '14:99:1E')'.
2025-03-27 11:14:38 [INFO] linescale3.classes.measurement.plot_force_vs_time_with_max_and_release: plot_force_vs_time_with_max_and_release for measurement: 'Measurement(id: '3', name: '14991E_23_03_22_14_55_03', sensor_id: '14:99:1E')'.
2025-03-27 11:14:39 [INFO] linescale3.classes.measurement.plot_force_vs_time_with_max_and_release: plot_force_vs_time_with_max_and_release for measurement: 'Measurement(id: '4', name: '14991E_23_03_22_15_04_05', sensor_id: '14:99:1E')'.
2025-03-27 11:14:39 [INFO] linescale3.classes.measur

### ANALYSE: Metadaten bzw. Zusammenfassung der Daten

'ls3_series.metadata_df' enthält für jeden Log (measurement_id) der 2 LineScales die Metadaten der Messung.

Anmerkung zu `release`: Für jeden Log von 14:BF:E6 (LineScale zur Messung der Vorspannung) wurde die Kraft in kN ermittelt, die kurz vor dem Release vorlag. Diese Kraft ist in den Plots (/ls3/plots/force_vs_time_1/) als grüne Linie markiert. Die entsprechende Funktion befindet sich in 'ls3/classes/measurement.py' und heißt 'get_release_force'.

Anmerkung zu `get_release_force`: Die Funktion filtert alle Kraftwerte über 1.0 kN (Parameter min_force=1.0). Innerhalb der gefilterten Werte wird ein Bereich ausgewählt, der 3 Sekunden vor dem Ende der Messung endet (Parameter distance_to_end_sec=3) und insgesamt 5 Sekunden lang ist (Parameter window_sec=5). Für diesen Zeitraum wird der Mittelwert (Mean) berechnet. Die release_force ist somit der Mittelwert eines 5-sekündigen Plateaus vor dem Release und wird nicht durch kurzzeitige Lastspitzen im Zugseil verfälscht.

In [7]:
ls3_metadata_df = ls3_series.metadata_df
ls3_metadata_df.head(3)

,measurement_name,sensor_id,datetime,measurement_id,unit,mode,rel_zero,speed,trig,stop,...,timing_correction_factor,datetime_start,datetime_end,duration,length,max,mean,median,min,release
0,14991E_23_03_22_14_41_10,14:99:1E,2022-03-23 14:41:10,1,kN,ABS,n/a,1280,0.8,0.0,...,0.9,2022-03-23 14:41:10,2022-03-23 14:41:26.199296875,16.199296,23040,"(4737, 1.48)",0.111598,0.06,"(36, 0.0)",NaN
1,14991E_23_03_22_14_47_59,14:99:1E,2022-03-23 14:47:59,2,kN,ABS,n/a,1280,0.8,0.0,...,0.9,2022-03-23 14:47:59,2022-03-23 14:48:15.199296875,16.199296,23040,"(4721, 1.44)",0.117817,0.06,"(135, 0.0)",NaN
2,14991E_23_03_22_14_55_03,14:99:1E,2022-03-23 14:55:03,3,kN,ABS,n/a,1280,0.8,0.0,...,0.9,2022-03-23 14:55:03,2022-03-23 14:55:19.199296875,16.199296,23040,"(3867, 1.33)",0.104375,0.06,"(81, 0.0)",NaN


## IMPORT: Datendokumentation laden

In [8]:
ls3_data_dict = ls3_series.create_data_dict()

2025-03-27 11:15:00 [INFO] linescale3.config.load_label_dict: Label dictionary loaded with 23 entries.


## ANALYSE: Datendokumentation für Metadaten ansehen

In [9]:
# In DataFrame umwandeln
ls3_data_dict_df =build_data_dict_df(ls3_data_dict)

# In Markdown umwandeln und anzeigen
md_text = ls3_data_dict_df.to_markdown(tablefmt="github")
display(Markdown(md_text))

|    | Variable                 | Kategorie   | Zeichen                | Deutsch             | Datentyp   | Einheit   | Beschreibung                                                 |
|----|--------------------------|-------------|------------------------|---------------------|------------|-----------|--------------------------------------------------------------|
|  0 | measurement_name         | ls3         | filename               | Dateiname           | object     | -         | Name der Log-Datei bzw. Serienbezeichnung der Messung        |
|  1 | sensor_id                | ls3         | sensor\_id             | Sensor ID           | object     | -         | ID des LineScale3 (Seriennummer)                             |
|  2 | measurement_id           | ls3         | $ID_{\mathrm{Log}}$    | LogNr.              | Int64      | -         | Mess-ID (interne LogNr. der LS3)                             |
|  3 | datetime                 | ls3         | $t$                    | Zeitstempel         | object     | -         | Startzeitpunkt der Messung laut Gerät                        |
|  4 | unit                     | ls3         | unit                   | Einheit             | object     | -         | Einheit der Kraftmessung (hier kN)                           |
|  5 | mode                     | ls3         | mode                   | Modus               | object     | -         | Messmodus, hier ABS für absolute Kraft                       |
|  6 | speed                    | ls3         | $f$                    | Abtastrate          | Int64      | Hz        | Messgeschwindigkeit in Hz                                    |
|  7 | trig                     | ls3         | $F_{\mathrm{trig}}$    | Triggerkraft        | float64    | kN        | Mindestkraft für Messbeginn                                  |
|  8 | pre                      | ls3         | $t_{\mathrm{pre}}$     | Vorlaufzeit         | Int64      | s         | Nachträglich ergänzter Zeitraum vor Trigger                  |
|  9 | catch                    | ls3         | $t_{\mathrm{catch}}$   | Aufzeichnungszeit   | Int64      | s         | Dauer der Datenerfassung nach Trigger                        |
| 10 | timing_correction_factor | ls3         | $t_{\mathrm{corr}}$    | Zeitkorrekturfaktor | float64    | -         | Korrekturfaktor für Zeit- und Dauerangaben (z. B. 0.9)       |
| 11 | duration                 | ls3         | $\Delta t$             | Messdauer           | float64    | s         | Gesamtdauer der Messung                                      |
| 12 | length                   | ls3         | $n$                    | Anzahl Messpunkte   | Int64      | -         | Anzahl erfasster Datenpunkte                                 |
| 13 | mean                     | ls3         | $\bar{F}$              | Mittlere Kraft      | float64    | kN        | Arithmetisches Mittel der gemessenen Kraft                   |
| 14 | median                   | ls3         | $F_{\mathrm{median}}$  | Median Kraft        | float64    | kN        | Median der gemessenen Kraft                                  |
| 15 | release                  | ls3         | $F_{\mathrm{release}}$ | Vorspannkraft Ist   | float64    | kN        | Tatsächlich realisierte Vorspannkraft im Zugseil bei Release |
| 16 | datetime_start           | ls3         | $t_{\mathrm{start}}$   | Startzeitpunkt      | object     | -         | Berechneter Startzeitpunkt der Messung                       |
| 17 | datetime_end             | ls3         | $t_{\mathrm{end}}$     | Endzeitpunkt        | object     | -         | Berechneter Endzeitpunkt der Messung                         |
| 18 | max                      | ls3         | $F_{\mathrm{max}}$     | Kraftmaximum        | object     | kN        | Index und Wert der maximalen Kraft                           |
| 19 | min                      | ls3         | $F_{\mathrm{min}}$     | Kraftminimum        | object     | kN        | Index und Wert der minimalen Kraft                           |
| 20 | rel_zero                 | ls3         | rel\_zero              | Nullpunkt relativ   | float64    | -         | Für ABS-Modus nicht relevant (hier NaN)                      |
| 21 | stop                     | ls3         | $t_{\mathrm{stop}}$    | Stopp-Kriterium     | float64    | s         | Zeitpunkt der Beendigung der Messung (fixe Dauer bei KS)     |
| 22 | total                    | ls3         | $t_{\mathrm{total}}$   | Messzeitraum gesamt | Int64      | s         | Gesamtlänge der Messung inkl. Pre-Trigger-Phase              |

## EXPORT: Daten exportieren für Weiterverarbeitung (.feather, .csv, .json)

In [10]:
# DataFrame als Feather
ls3_metadata_df.to_feather(data_export_directory / "ls3.feather")
ls3_metadata_df.to_csv(data_export_directory / "ls3.csv", sep=";", index=False, encoding="utf-8")

# Dict als JSON (UTF-8, sauber eingerückt)
with open(data_export_directory / "ls3_data_dict.json", "w", encoding="utf-8") as f:
    json.dump(ls3_data_dict, f,  indent=4, ensure_ascii=False)

In [11]:
# Feather laden

# ls3_metadata_df = pd.read_feather(data_export_directory / "ls3_metadata.feather")
# 
# # JSON laden
# with open(data_export_directory / "ls3_data_dict.json", "r", encoding="utf-8") as f:
#     ls3_data_dict = json.load(f)

## LATEX-EXPORT: Daten und Datendokumentation als Latex-Tabelle exportieren (.tex)

In [12]:
ls3_data_dict_df = build_data_dict_df(ls3_data_dict, escape_index=True, select_latex_fields=True)
latex_string = ls3_data_dict_df.to_latex(index=False, escape=False)
caption = "Feldversuch 2 - LS3 Daten Dokumentation"

save_latex_table(latex_string, caption, latex_export_directory)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ls3_daten_dokumentation.tex
